# Project 3

Import some libraries and the datasets

In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from ISLP import load_data, confusion_table
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (accuracy_score, log_loss)
from sklearn.ensemble import (RandomForestRegressor, RandomForestClassifier)
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics


# main dataset
data_index_2 = pd.read_csv('../Project_datasets/data_index_2.csv')
# no idea what this is
data_index_example = pd.read_csv('../Project_datasets/data_index - example.csv')
LPJ_GUESS_output = pd.read_csv('../Project_datasets/LPJ-GUESS_output_BERN1.csv')
# precipitation by day and statistics
Predaymean = pd.read_csv('../Project_datasets/Predaymean1961_1990.csv')
Predaymean_statistics = pd.read_csv('../Project_datasets/Predaymean1961_1990_statics.csv')
# maximum temperature by day and statistics
Tmaxdaymean = pd.read_csv('../Project_datasets/Tmaxdaymean1961_1990.csv')
Tmaxdaymean_statistics = pd.read_csv('../Project_datasets/Tmaxdaymean1961_1990_statics.csv')
# minimum temperature by day and statistics
Tmindaymean = pd.read_csv('../Project_datasets/Tmindaymean1961_1990.csv')
Tmindaymean_statistics = pd.read_csv('../Project_datasets/Tmindaymean1961_1990_statics.csv')
# mean temperature by day and statistics
Tmpdaymean = pd.read_csv('../Project_datasets/Tmpdaymean1961_1990.csv')
Tmpdaymean_statistics = pd.read_csv('../Project_datasets/Tmpdaymean1961_1990_statics.csv')
# shortwave radiation flux
Tswrfdaymean = pd.read_csv('../Project_datasets/Tswrfdaymean1961_1990.csv')
Tswrfdaymean_statistics = pd.read_csv('../Project_datasets/Tswrfdaymean1961_1990_statics.csv')

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


Show the first entries

In [3]:
print(f'shape: {data_index_2.shape}')
data_index_2.head()

shape: (59191, 84)


,Unnamed: 0,Lon,Lat,clay,silt,sand,orgC,CN,pH,cellfraction,...,VegC,LitterC,SoilC,Biome_Cmax,Biome_LAI,Biome_obs,GFED-region,Pan_2007,ISO3,UN
0,0,-179.75,71.25,0.08,0.37,0.55,0.020,11.0,5.9,0.482,...,0.0,0.000,0.000,13,13,17,10,'Russia','RUS',643
1,1,-179.75,68.75,0.20,0.48,0.32,0.031,17.0,6.3,0.753,...,0.0,0.003,0.015,13,11,17,10,'Russia','RUS',643
2,2,-179.75,68.25,0.20,0.48,0.32,0.031,17.0,6.3,0.447,...,0.0,0.006,0.022,13,11,17,10,'Russia','RUS',643
3,3,-179.75,67.75,0.20,0.48,0.32,0.031,17.0,6.3,0.526,...,0.0,0.002,0.020,13,11,17,10,'Russia','RUS',643
4,4,-179.75,67.25,0.20,0.48,0.32,0.031,17.0,6.3,0.422,...,0.0,0.003,0.016,13,11,17,10,'Russia','RUS',643


In [4]:
print(f'shape: {data_index_example.shape}')
data_index_example.head()

shape: (38470, 80)


,GridNo,Lon,Lat,clay,silt,sand,orgC,CN,pH,cellfraction,...,NPP,SoilR,MaxBiomeCmax,MaxBiomeLAI,VegC,LitterC,SoilC,Biome_Cmax,Biome_LAI,Biome_obs
0,0.0,-179.75,71.25,0.08,0.37,0.55,0.020,11.0,5.9,0.482,...,0.0,0.0,0.0,0.0000,0.0,0.000,0.000,13.0,13.0,17.0
1,1.0,-179.75,68.75,0.20,0.48,0.32,0.031,17.0,6.3,0.753,...,0.0,0.0,0.0,0.0003,0.0,0.003,0.015,13.0,11.0,17.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(f'shape: {LPJ_GUESS_output.shape}')
LPJ_GUESS_output.head()

shape: (59191, 12)


,Lon,Lat,NPP,SoilR,MaxBiomeCmax,MaxBiomeLAI,VegC,LitterC,SoilC,Biome_Cmax,Biome_LAI,Biome_obs
0,39.75,-1.25,0.429,0.390,0.449,1.9821,1.225,0.758,5.941,8,12,12
1,150.25,-34.25,0.554,0.451,6.883,3.3174,6.953,3.221,10.566,7,7,6
2,-63.75,82.75,0.000,0.000,0.000,0.0000,0.000,0.003,0.002,13,13,17
3,59.25,30.75,0.043,0.042,0.090,0.2146,0.127,0.084,0.543,7,11,14
4,24.25,27.75,0.000,0.000,0.000,0.0000,0.000,0.000,0.000,13,13,17


In [6]:
print(f'shape: {Predaymean.shape}')
Predaymean.head()

shape: (59191, 367)


,Lon,Lat,Day1,Day2,Day3,Day4,Day5,Day6,Day7,Day8,...,Day356,Day357,Day358,Day359,Day360,Day361,Day362,Day363,Day364,Day365
0,39.75,-1.25,0.634400,0.838580,1.373300,1.487800,0.948640,0.705560,0.790570,0.69274,...,1.260600,1.011600,2.676200,2.146300,0.83299,0.97611,1.092800,2.015100,1.220200,1.064100
1,150.25,-34.25,3.176800,3.282800,4.271300,2.735100,3.118300,1.863200,2.245400,1.81270,...,1.243100,2.901500,1.580400,2.788500,3.62270,2.12350,2.546700,2.624700,1.521800,2.259400
2,-63.75,82.75,0.191720,0.553370,0.559100,0.350270,0.414340,0.239620,0.215600,0.39415,...,0.286600,0.187410,0.352480,0.303120,0.37439,0.29827,0.399200,0.162680,0.180370,0.251560
3,59.25,30.75,0.272920,0.576200,1.529900,0.475970,0.809060,0.772860,0.773340,0.12219,...,0.315400,0.223630,0.009584,0.004932,0.65535,1.25430,0.680690,0.534400,0.225750,0.281360
4,24.25,27.75,0.012822,0.044511,0.027133,0.003318,0.007306,0.014162,0.085964,0.01347,...,0.029099,0.040272,0.030631,0.000518,0.00058,0.12653,0.010889,0.002139,0.003662,0.009687


In [7]:
print(f'shape: {Predaymean_statistics.shape}')
Predaymean_statistics.head()

shape: (59191, 14)


,Lon,Lat,SpringMean,SpringMedian,SpringStd,SummerMean,SummerMedian,SummerStd,FallMean,FallMedian,FallStd,WinterMean,WinterMedian,WinterStd
0,39.75,-1.25,0.870380,0.693390,0.641700,1.622500,1.30490,1.11930,0.528780,0.450650,0.315910,2.049600,1.836000,1.289100
1,150.25,-34.25,3.251700,2.971600,1.559000,2.418500,2.07100,1.51770,1.725700,1.331900,1.054600,2.547500,2.358200,1.253600
2,-63.75,82.75,0.234270,0.209640,0.139210,0.333730,0.31885,0.18320,0.758070,0.735040,0.338730,0.325480,0.296160,0.154960
3,59.25,30.75,0.715820,0.592520,0.423350,0.136010,0.03079,0.17825,0.005847,0.000000,0.020282,0.180990,0.045619,0.278850
4,24.25,27.75,0.045284,0.015422,0.070006,0.061916,0.01050,0.10039,0.004480,0.000016,0.022542,0.028823,0.006414,0.055641


In [8]:
print(f'shape: {Tmaxdaymean.shape}')
Tmaxdaymean.head()

shape: (59191, 367)


,Lon,Lat,Day1,Day2,Day3,Day4,Day5,Day6,Day7,Day8,...,Day356,Day357,Day358,Day359,Day360,Day361,Day362,Day363,Day364,Day365
0,39.75,-1.25,302.63,302.91,302.83,302.43,302.76,302.99,302.95,303.04,...,302.96,303.14,302.65,302.49,302.66,302.87,302.73,302.76,302.99,302.98
1,150.25,-34.25,296.43,296.09,295.97,294.85,294.73,295.55,295.77,296.80,...,296.44,296.16,295.83,296.28,295.14,295.86,296.45,296.14,296.54,294.66
2,-63.75,82.75,244.72,245.60,245.89,245.11,244.98,244.50,243.61,243.90,...,246.04,246.82,247.53,247.26,246.58,246.02,246.67,247.49,247.69,246.57
3,59.25,30.75,284.75,284.92,285.61,286.23,285.53,285.78,286.21,285.91,...,287.97,288.04,287.38,286.80,287.24,287.00,286.86,286.56,286.61,286.65
4,24.25,27.75,290.05,289.76,289.50,289.10,289.05,289.41,289.26,288.84,...,290.04,289.73,289.58,289.62,289.48,289.77,289.97,290.24,290.17,289.90


## Section 3

Choose countries. Random.org gave us the biomes
- 16: arid shrub and
- 15: xeric shrub

In [66]:
biome_list = [17,14] # our chosen ones

fineness = 'ISO3' #'Pan_2007'
biome_countries = data_index_2[fineness].drop_duplicates()
for biome_nbr in biome_list:
    # print(data_index_2.loc[data_index_2.loc[data_index_2['Pan_2007'].isin(biome_countries)]['Biome_obs'].isin([biome_nbr])]['Pan_2007'].drop_duplicates())
    relevant_countries = data_index_2.loc[data_index_2[fineness].isin(biome_countries)]
    biome_countries = relevant_countries.loc[relevant_countries['Biome_obs']==biome_nbr][fineness].drop_duplicates()
print(f'the following countries are relevant: {biome_countries}')
# select two random countries
chosen_countries = [stats.randint.rvs(0,len(biome_countries)-1) for i in range(2)]
print(chosen_countries)
chosen_countries = [biome_countries.iloc[chosen_county] for chosen_county in chosen_countries]
for chosen_country in chosen_countries:
    print(f'We choose 2 random countries: {chosen_country}')
# stats.randint.rvs(0,len(biome_countries)-1)
# set countries to Egypt and China
chosen_countries = ['EGY','CHN']

the following countries are relevant: 2258     'CAN'
2862     'USA'
10015    'PER'
12200    'CHL'
12563    'ARG'
18638       []
18655    'MRT'
18871    'MAR'
19022    'MLI'
19355    'DZA'
20857    'NER'
22086    'CHE'
22098    'TUN'
22508    'ITA'
22640    'LBY'
23188    'NAM'
23710    'TCD'
27259    'SDN'
27616    'EGY'
33089    'SOM'
39973    'CHN'
42776    'RUS'
Name: ISO3, dtype: object
[0, 18]
We choose 2 random countries: 'CAN'
We choose 2 random countries: 'EGY'


Thus we choose Africa to train and China as test

In [48]:
# NaN appears in table, see where it occurs
print(list(data_index_2.loc[:,data_index_2.isna().any()]))
data_index_2[data_index_2.isna().any(axis=1)]

,Unnamed: 0,Lon,Lat,clay,silt,sand,orgC,CN,pH,cellfraction,...,VegC,LitterC,SoilC,Biome_Cmax,Biome_LAI,Biome_obs,GFED-region,Pan_2007,ISO3,UN
59182,59182,179.75,71.25,0.08,0.37,0.55,0.02,NaN,NaN,NaN,...,0.0,0.0,0.001,13,13,17,0,[],[],0


In [65]:
# restrict to two biomes, restrict to country codes, remove any rows with NaN
relevant_data = data_index_2.loc[data_index_2['Biome_obs'].isin(biome_list)] # restrict to two biomes
relevant_data = relevant_data.loc[relevant_data['ISO3'].isin(chosen_countries)] # restrict to two countries
relevant_data = relevant_data[~(relevant_data.isna().any(axis=1))] # delete rows with NaN
X = relevant_data.drop(columns=['Biome_obs','Biome_LAI','Biome_Cmax','Lon','Lat','Pan_2007','ISO3','UN'])
feature_names = list(X.columns)

Y = relevant_data['Biome_obs']
Y = Y==biome_list[0] # give True if it is the first biome in the list

clf = RandomForestClassifier(random_state=0)
clf.fit(X, Y)
(X_train,
 X_test,
 Y_train,
 Y_test) = skm.train_test_split(X,
                                   Y,
                                   test_size=0.3,
                                   random_state=0)
# kfold3 = skm.KFold(3,
#                   random_state=1,
#                   shuffle=True) # let's try fewer fold to save running time
# n_depths_range = np.linspace(1, 10, 10).astype('int')
# n_estimators_range = np.linspace(10, 100, 10).astype('int') # CV number of trees (estimators) 10, 20, ..., 100

# params = {
#     'max_depth': n_depths_range, # let's tune for only one parameters to save running time
#     'n_estimators': n_estimators_range,
#      }
# rfc_gscv = GridSearchCV(clf, param_grid = params, scoring = "accuracy",
#                              cv = kfold3 )
# # Fit the model
# model_rfc = rfc_gscv.fit(X_train, Y_train)